In [14]:
import cameo
import pandas as pd
from rdkit import Chem
import os
import json

In [3]:
struct_data = pd.read_excel('../input/data/database_struct_mets_iML1515.xlsx',index_col=0)
print(struct_data.shape)
struct_data.head()

(1005, 4)


,name,formula,iJO1366,inchi
10fthf,10-Formyltetrahydrofolate,C20H21N7O7,10fthf,InChI=1S/C20H23N7O7/c21-20-25-16-15(18(32)26-2...
12dgr120,"1,2-Diacyl-sn-glycerol (didodecanoyl, n-C12:0)",C27H52O5,12dgr120,InChI=1S/C27H52O5/c1-3-5-7-9-11-13-15-17-19-21...
12dgr140,"1,2-Diacyl-sn-glycerol (ditetradecanoyl, n-C14:0)",C31H60O5,12dgr140,InChI=1S/C31H60O5/c1-3-5-7-9-11-13-15-17-19-21...
12dgr141,"1,2-Diacyl-sn-glycerol (ditetradec-7-enoyl, n-...",C31H56O5,12dgr141,InChI=1S/C31H56O5/c1-3-5-7-9-11-13-15-17-19-21...
12dgr160,"1,2-Diacyl-sn-glycerol (dihexadecanoyl, n-C16:0)",C35H68O5,12dgr160,InChI=1S/C35H68O5/c1-3-5-7-9-11-13-15-17-19-21...


In [4]:
struct_data['inchi'].isnull().value_counts()

False    1005
Name: inchi, dtype: int64

In [23]:
def inchitosmiles(inchi):
    return Chem.MolToSmiles(Chem.MolFromInchi(inchi))
struct_data2 = pd.DataFrame().assign(smiles=struct_data['inchi'].apply(inchitosmiles))

In [24]:
struct_data2.head()

,smiles
10fthf,N=c1nc(O)c2c([nH]1)NCC(CN(C=O)c1ccc(C(=O)N[C@@...
12dgr120,CCCCCCCCCCCC(=O)OC[C@H](CO)OC(=O)CCCCCCCCCCC
12dgr140,CCCCCCCCCCCCCC(=O)OC[C@H](CO)OC(=O)CCCCCCCCCCCCC
12dgr141,CCCCCC/C=C\CCCCCC(=O)OCC(CO)OC(=O)CCCCC/C=C\CC...
12dgr160,CCCCCCCCCCCCCCCC(=O)OC[C@H](CO)OC(=O)CCCCCCCCC...


In [25]:
struct_data2.to_csv('/media/frek/Storage/projects/chemocobra/static/chemoinfo/smiles_iML1515.csv')

In [32]:
def get_model(model_id):
    STATIC_DIR = '/media/frek/Storage/projects/chemocobra/static/'
    model_path = os.path.join(STATIC_DIR,'metabolic','models',model_id+'.json')
    with open(os.path.join(model_path)) as f:
        model = json.load(f)
    model['id'] = model_id
    return model

In [16]:
model_json = get('iML1515_PKT')

In [18]:
model_json.keys()

dict_keys(['reactions', 'metabolites', 'genes', 'id', 'name', 'compartments', 'version'])

In [20]:
len(model_json['metabolites'])

1887

In [22]:
model_json['metabolites'][0]

{'id': '10fthf_c',
 'name': '10-Formyltetrahydrofolate',
 'compartment': 'c',
 'charge': -2,
 'formula': 'C20H21N7O7'}

In [26]:
model_json['genes'][0]

{'id': 'b0002', 'name': 'thrA'}

In [28]:
model_json['reactions'][15]

{'id': '23CCMPtex',
 'name': '23cCMP transport via diffusion (extracellular to periplasm)',
 'metabolites': {'23ccmp_e': -1.0, '23ccmp_p': 1.0},
 'lower_bound': -1000.0,
 'upper_bound': 1000.0,
 'gene_reaction_rule': 'b0241 or b0929 or b1377 or b2215'}

## Add data to models

## Genes

In [81]:
genes_data = pd.read_csv('../input/df_genes_modsouch.csv',index_col=0,sep=';')
print(genes_data.shape)
genes_data.head(2)

(4454, 8)


,ecocyc_id,synonyms,products,n_prods,ko_obs,url,uniprot,class
b0399,EG10728,phoB phoRc phoT,PHOSPHO-PHOB PHOB-MONOMER,2.0,OBS0-49 OBS0-37 OBS0-33 OBS0-44,http://ecocyc.org/gene?orgid=ECOLI&id=EG10728,NaN,repressor
b2839,EG10551,lysR,PD00360,1.0,OBS0-49 OBS0-37 OBS0-33,http://ecocyc.org/gene?orgid=ECOLI&id=EG10551,P03030,repressor


In [ ]:
# For GENES add: ecocyc_id an synonyms --> only for iML1515
for mid in models_iML1515:
    model = models[mid]
    for g in model['genes']:
        if g['id'] in genes_data.index:
            g['ecocyc_id'] = genes_data.loc[g['id'],'ecocyc_id']
            g['synonyms'] = genes_data.loc[g['id'],'synonyms']

In [95]:
models_iML1515 = ['iML1515_PKT', 'iML1515', 'iML1515_PKT_test']
models = {}
model_dir = '/media/frek/Storage/projects/chemocobra/static/metabolic/models/'
for file_model in [f for f in os.listdir(model_dir) if f.endswith('.json') and 'old' not in f]:
    with open((model_dir+file_model)) as f:
        models[file_model.replace('.json','')] = json.load(f)
    #models[file_model.replace('.json','')] = cameo.load_model(model_dir+file_model)
print(models.keys())

dict_keys(['iML1515_PKT', 'iML1515', 'iJO1366', 'RECON1', 'iHN637', 'iML1515_PKT_test'])


## Metabolites

In [78]:
print(struct_data2.shape)
struct_data2.head()

(1005, 1)


,smiles
10fthf,N=c1nc(O)c2c([nH]1)NCC(CN(C=O)c1ccc(C(=O)N[C@@...
12dgr120,CCCCCCCCCCCC(=O)OC[C@H](CO)OC(=O)CCCCCCCCCCC
12dgr140,CCCCCCCCCCCCCC(=O)OC[C@H](CO)OC(=O)CCCCCCCCCCCCC
12dgr141,CCCCCC/C=C\CCCCCC(=O)OCC(CO)OC(=O)CCCCC/C=C\CC...
12dgr160,CCCCCCCCCCCCCCCC(=O)OC[C@H](CO)OC(=O)CCCCCCCCC...


In [187]:
# Load data metabolites BIGG
met_bigg = pd.read_csv('/media/frek/Storage/Data/BIGG/bigg_models_metabolites.txt',sep='\t').drop_duplicates(subset=['universal_bigg_id'], keep='first')
met_bigg = met_bigg.set_index('universal_bigg_id')[['name','database_links']]
for idx in met_bigg.index:
    mnx = [':'.join(x.split(':')[1:]) for x in str(met_bigg.loc[idx,'database_links']).split('; ') if 'MNX' in str(x).split(':')[0]]
    if len(mnx)>0:
        met_bigg.loc[idx,'MNX'] = mnx[0].split('/')[-1]
    else:
        met_bigg.loc[idx,'MNX'] = ''
print(met_bigg.shape)
met_bigg.head()

(7340, 3)


,name,database_links,MNX
universal_bigg_id,,,
10fthf,10-Formyltetrahydrofolate,Reactome: http://www.reactome.org/content/deta...,MNXM237
12dgr120,"1,2-Diacyl-sn-glycerol (didodecanoyl, n-C12:0)",MetaNetX (MNX) Chemical: http://identifiers.or...,MNXM4939
12dgr140,"1,2-Diacyl-sn-glycerol (ditetradecanoyl, n-C14:0)",MetaNetX (MNX) Chemical: http://identifiers.or...,MNXM146479
12dgr161,"1,2-Diacyl-sn-glycerol (dihexadec-9-enoyl, n-C...",MetaNetX (MNX) Chemical: http://identifiers.or...,MNXM3774
12dgr180,"1,2-Diacyl-sn-glycerol (dioctadecanoyl, n-C18:0)",MetaNetX (MNX) Chemical: http://identifiers.or...,MNXM4217


In [144]:
# Load data metabolites MNX
met_mnx = pd.read_csv('/media/frek/Storage/Data/MNX/chem_prop.tsv',sep='\t',comment='#',header=None)
met_mnx = met_mnx.drop_duplicates(subset=[0])[[0,1,5,6]].set_index(0)
met_mnx.columns = ['name','inchi','smiles']
print(met_mnx.shape)
met_mnx.head()

(694311, 3)


,name,inchi,smiles
0,,,
BIOMASS,Biomass,NaN,NaN
MNXM0,molecular entity,NaN,NaN
MNXM01,H(+),InChI=1S/p+1,[H+]
MNXM1,H(+),InChI=1S/p+1,[H+]
MNXM2,H2O,InChI=1S/H2O/h1H2,[H]O[H]


In [188]:
# Merge MNX with BIGG
met_bigg = met_bigg.merge(met_mnx[['inchi','smiles']],how='left',left_on='MNX',right_index=True)
print(met_bigg.shape)
met_bigg.head(3)

(7340, 5)


,name,database_links,MNX,inchi,smiles
universal_bigg_id,,,,,
10fthf,10-Formyltetrahydrofolate,Reactome: http://www.reactome.org/content/deta...,MNXM237,InChI=1S/C20H23N7O7/c21-20-25-16-15(18(32)26-2...,[H]C(=O)N(C[C@H]1CNc2nc(N)[nH]c(=O)c2N1)c1ccc(...
12dgr120,"1,2-Diacyl-sn-glycerol (didodecanoyl, n-C12:0)",MetaNetX (MNX) Chemical: http://identifiers.or...,MNXM4939,NaN,NaN
12dgr140,"1,2-Diacyl-sn-glycerol (ditetradecanoyl, n-C14:0)",MetaNetX (MNX) Chemical: http://identifiers.or...,MNXM146479,NaN,NaN


In [189]:
## See if metabolites in models are indeed present in met_bigg.index --> OK =)
for mid in models:
    for met in models[mid]['metabolites']:
        met_id = '_'.join(met['id'].split('_')[:-1])
        if met_id not in met_bigg.index:
            print(mid,met_id)

iML1515_PKT acphe
iML1515_PKT_test acphe


In [190]:
## verify structures
for met_id in met_bigg.index:
    # verify inchi
    inchi_ok = False
    try:
        mol = Chem.MolFromInchi(met_bigg.loc[met_id,'inchi'])
    except:
        pass
    else:
        if mol is not None:
            inchi_ok = True
            
    # verify smiles
    smiles_ok = False
    try:
        mol = Chem.MolFromSmiles(met_bigg.loc[met_id,'smiles'])
    except:
        pass
    else:
        if mol is not None:
            smiles_ok = True
    if smiles_ok:
        met_bigg.loc[met_id,'smiles_ok'] = met_bigg.loc[met_id,'smiles']
    elif inchi_ok:
        mol = Chem.MolFromInchi(met_bigg.loc[met_id,'inchi'])
        met_bigg.loc[met_id,'smiles_ok'] = Chem.MolToSmiles(mol)
    else:
        met_bigg.loc[met_id,'smiles_ok'] = ''

In [191]:
met_bigg['smiles_ok'].apply(lambda x:x!='').value_counts()

False    5242
True     2098
Name: smiles_ok, dtype: int64

In [180]:
print(met_id, struct_data2.loc[met_id])
met_bigg.loc[met_id]

10fthf smiles    N=c1nc(O)c2c([nH]1)NCC(CN(C=O)c1ccc(C(=O)N[C@@...
Name: 10fthf, dtype: object


name                                 10-Formyltetrahydrofolate
MNX                                                    MNXM237
smiles_ok    [H]C(=O)N(C[C@H]1CNc2nc(N)[nH]c(=O)c2N1)c1ccc(...
Name: 10fthf, dtype: object

In [192]:
## Finally, data for ecoli is better:
for met_id in struct_data2.index:
    if struct_data2.loc[met_id,'smiles'] != '':
        met_bigg.loc[met_id,'smiles_ok'] = struct_data2.loc[met_id,'smiles']

In [193]:
met_bigg['smiles_ok'].apply(lambda x:x!='').value_counts()

False    5107
True     2234
Name: smiles_ok, dtype: int64

In [197]:
met_bigg = met_bigg[['name','MNX','smiles_ok']]
met_bigg['MNX'] = met_bigg['MNX'].fillna('')
met_bigg.head()

,name,MNX,smiles_ok
universal_bigg_id,,,
10fthf,10-Formyltetrahydrofolate,MNXM237,N=c1nc(O)c2c([nH]1)NCC(CN(C=O)c1ccc(C(=O)N[C@@...
12dgr120,"1,2-Diacyl-sn-glycerol (didodecanoyl, n-C12:0)",MNXM4939,CCCCCCCCCCCC(=O)OC[C@H](CO)OC(=O)CCCCCCCCCCC
12dgr140,"1,2-Diacyl-sn-glycerol (ditetradecanoyl, n-C14:0)",MNXM146479,CCCCCCCCCCCCCC(=O)OC[C@H](CO)OC(=O)CCCCCCCCCCCCC
12dgr161,"1,2-Diacyl-sn-glycerol (dihexadec-9-enoyl, n-C...",MNXM3774,CCCCCC/C=C\CCCCCCCC(=O)OC[C@H](CO)OC(=O)CCCCCC...
12dgr180,"1,2-Diacyl-sn-glycerol (dioctadecanoyl, n-C18:0)",MNXM4217,CCCCCCCCCCCCCCCCCC(=O)OC[C@H](CO)OC(=O)CCCCCCC...


In [200]:
met_bigg.to_csv('/media/frek/Storage/projects/chemocobra/static/metabolic/met_bigg_structs_mnx.csv')

In [199]:
## Add this information to metabolites in models:
for model_id in models:
    for met in models[model_id]['metabolites']:
        m_id = '_'.join(met['id'].split('_')[:-1])
        if m_id in met_bigg.index:
            met['MNX'] = met_bigg.loc[m_id,'MNX']
            met['smiles'] = met_bigg.loc[m_id,'smiles_ok']

## Reactions

In [234]:
for m_id in models:
    genes_model = {g['id']:g['name'] for g in models[m_id]['genes']}
    for r in models[m_id]['reactions']:
        genes_reaction = []
        for g_id in genes_model:
            if g_id in r['gene_reaction_rule']:
                if genes_model[g_id] == '':
                    genes_reaction.append(g_id)
                else:
                    genes_reaction.append(g_id+'('+genes_model[g_id]+')')
        r['genes'] = ', '.join(genes_reaction)

## Sauvegarder modèle

In [237]:
os.system('mkdir -p '+STATIC_DIR+'metabolic/models-new')
for m_id in models:
    with open(STATIC_DIR+'metabolic/models-new/'+m_id+'.json','w') as f:
        json.dump(models[m_id],f)